In [14]:
%matplotlib inline

import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as mse

from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np

from tqdm.notebook import tqdm

In [3]:
from pathlib import PurePath

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

In [7]:
NUM_FRAMES = 344   # number of timestep. Before was 9!


torch.manual_seed(0)

Our simulation managed to create `NUM_FRAMES` in csv forms

In [8]:
data_paths = [PurePath("new_data", f"data_._{i}.csv") for i in range(0,NUM_FRAMES)]
# dfs = [0]*NUM_FRAMES
# for i in range(0,NUM_FRAMES):

#     dfs[i]=pd.read_csv(str(data_path[i]))

In [11]:
dfs_0 = pd.read_csv(str(data_paths[0]))
dfs_0
# p is related to diffusion rate of H2 through metal lattice

,Time,Points:0,Points:1,Points:2,U:0,U:1,U:2,p
0,0.00004,0.023410,0.000500,0.415070,0.000000,0.000000,0.000,133.810
1,0.00004,0.020414,-0.000500,0.255010,0.000000,0.000000,0.000,382.260
2,0.00004,0.024436,0.000449,0.011233,-0.022317,-0.000197,17.772,762.260
3,0.00004,0.014274,-0.000449,0.419120,0.001344,-0.002953,17.661,127.700
4,0.00004,-0.011071,-0.000371,0.121100,-0.019496,-0.002143,20.441,591.670
...,...,...,...,...,...,...,...,...
984916,0.00004,0.017537,0.000371,0.268350,0.027584,-0.002228,20.377,361.710
984917,0.00004,0.027954,-0.000401,0.492970,-0.113740,0.027516,19.616,12.412
984918,0.00004,-0.002297,-0.000371,0.310220,-0.015338,0.017929,20.431,296.030
984919,0.00004,-0.012110,0.000449,0.287680,-0.031089,0.005600,17.710,331.210


5 snapshot is for training data

We concat every `p` of the first 5 frames and try to predict the rest of the frames. Matrix is a long one, has the shape of `(m=5, n=272k)`
$$\begin{bmatrix} C_{1,1} & C_{1,2} & \cdots & C_{1,n} \\ C_{2,1} & C_{2,2} & \cdots & C_{2,n} \\ \vdots & \vdots & \vdots & \vdots \\ C_{m,1} & C_{m,2} & \cdots & C_{m,n} \end{bmatrix}$$

In [12]:
dfs_1 = pd.read_csv(str(data_paths[1]))
dfs_1

,Time,Points:0,Points:1,Points:2,U:0,U:1,U:2,p
0,0.0001,0.023410,0.000500,0.415070,0.000000,0.000000,0.000,88.5500
1,0.0001,0.020414,-0.000500,0.255010,0.000000,0.000000,0.000,255.1500
2,0.0001,0.024436,0.000449,0.011233,0.028363,0.002333,15.109,509.4300
3,0.0001,0.014274,-0.000449,0.419120,0.035358,-0.002790,15.068,84.3860
4,0.0001,-0.011071,-0.000371,0.121100,-0.069954,0.001703,20.613,395.1300
...,...,...,...,...,...,...,...,...
984916,0.0001,0.017537,0.000371,0.268350,0.004827,-0.005425,20.518,241.3000
984917,0.0001,0.027954,-0.000401,0.492970,-0.094353,0.043290,19.115,7.3247
984918,0.0001,-0.002297,-0.000371,0.310220,0.076176,0.012086,20.582,197.6300
984919,0.0001,-0.012110,0.000449,0.287680,-0.027800,0.004144,15.117,221.1700


In [15]:
# the points position are the same in the same order in all frames
p_data = dfs_0['p']

for i in tqdm(range(1,NUM_FRAMES)):
    _ = pd.read_csv(str(data_paths[i]))
    pd.testing.assert_frame_equal(dfs_0[["Points:0", "Points:1", "Points:2"]], _[["Points:0", "Points:1", "Points:2"]])
    p_data = pd.concat([p_data, _['p']], axis=1)

  0%|          | 0/343 [00:00<?, ?it/s]

All points coordinates are the same accross time

In [16]:
p_data

,p,p,p,p,p,p,p,p,p,p,...,p,p,p,p,p,p,p,p,p,p
0,133.810,88.5500,61.8800,51.3900,45.9870,42.6070,40.2300,38.5460,37.5610,37.3210,...,44.4830,44.483,44.4830,44.483,44.4830,44.4830,44.4830,44.4830,3.99910,44.4830
1,382.260,255.1500,178.1100,147.4700,131.7100,121.9100,115.1100,110.3800,107.6000,106.9200,...,127.3700,127.370,127.3700,127.370,127.3700,127.3700,127.3700,127.3700,11.45100,127.3700
2,762.260,509.4300,355.4600,294.0100,262.3800,242.8000,229.3800,220.1400,214.8200,213.5800,...,252.5600,252.560,252.5600,252.560,252.5600,252.5600,252.5600,252.5600,22.70500,252.5600
3,127.700,84.3860,58.9740,48.9890,43.8430,40.6190,38.3530,36.7510,35.8140,35.5890,...,42.4210,42.421,42.4210,42.421,42.4210,42.4210,42.4210,42.4210,3.81370,42.4210
4,591.670,395.1300,275.7700,228.2100,203.7300,188.5600,178.1100,170.8800,166.7100,165.7800,...,197.9200,197.920,197.9200,197.920,197.9200,197.9200,197.9200,197.9200,17.79300,197.9200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984916,361.710,241.3000,168.4500,139.4700,124.5600,115.2800,108.8500,104.3800,101.7700,101.1300,...,120.3600,120.360,120.3600,120.360,120.3600,120.3600,120.3600,120.3600,10.82000,120.3600
984917,12.412,7.3247,5.2495,4.5522,4.1665,3.8946,3.6573,3.4477,3.3057,3.2698,...,4.0471,4.047,4.0471,4.047,4.0471,4.0471,4.0471,4.0472,0.36383,4.0471
984918,296.030,197.6300,138.0000,114.3200,102.1300,94.5450,89.2740,85.5920,83.4400,82.9110,...,98.7990,98.799,98.7990,98.799,98.7990,98.7990,98.7990,98.7990,8.88200,98.7990
984919,331.210,221.1700,154.4100,127.8700,114.2100,105.7100,99.8210,95.7150,93.3240,92.7450,...,110.4700,110.470,110.4700,110.470,110.4700,110.4700,110.4700,110.4700,9.93150,110.4700


In [45]:
train_index = int(p_data.shape[1] * .5)
val_index = train_index + int(p_data.shape[1] * .1)

In [29]:
p_values = p_data.values

In [46]:
train_data = p_values[:,:train_index]
val_data = p_values[:,train_index:val_index]
test_data = p_values[:,val_index:]

In [47]:
train_data.shape

(984921, 172)

In [48]:
val_data.shape

(984921, 34)

In [49]:
test_data.shape

(984921, 138)

In [50]:
test_data

array([[ 44.483 ,  44.483 ,  44.483 , ...,  44.483 ,   3.9991,  44.483 ],
       [127.37  , 127.37  , 127.37  , ..., 127.37  ,  11.451 , 127.37  ],
       [252.56  , 252.56  , 252.56  , ..., 252.56  ,  22.705 , 252.56  ],
       ...,
       [ 98.799 ,  98.799 ,  98.799 , ...,  98.799 ,   8.882 ,  98.799 ],
       [110.47  , 110.47  , 110.47  , ..., 110.47  ,   9.9315, 110.47  ],
       [ 40.592 ,  40.592 ,  40.592 , ...,  40.592 ,   3.6492,  40.592 ]])

In [51]:
train_data

array([[133.81 ,  88.55 ,  61.88 , ...,  44.483,  44.483,  44.483],
       [382.26 , 255.15 , 178.11 , ..., 127.37 , 127.37 , 127.37 ],
       [762.26 , 509.43 , 355.46 , ..., 252.56 , 252.56 , 252.56 ],
       ...,
       [296.03 , 197.63 , 138.   , ...,  98.799,  98.799,  98.799],
       [331.21 , 221.17 , 154.41 , ..., 110.47 , 110.47 , 110.47 ],
       [121.58 ,  80.613,  56.355, ...,  40.592,  40.592,  40.592]])

In [53]:
import pickle

with open('train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f)

In [54]:
with open('val_data.pkl', 'wb') as f:
    pickle.dump(val_data, f)
with open('test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)